In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import numpy as np
from snorkel import SnorkelSession
session = SnorkelSession()

In [ ]:
from snorkel.models import candidate_subclass
from snorkel.annotations import load_gold_labels

Indicator = candidate_subclass('Indicator', ['trend'], ['indicator'])
L_gold_dev = load_gold_labels(session, annotator_name='gold', split=1)

In [ ]:
import re
from snorkel.lf_helpers import (
    get_left_tokens, get_right_tokens, get_between_tokens,
    get_text_between, get_tagged_text,get_sent_candidate_spans
)

In [ ]:
import re 
import spacy
nlp = spacy.load('en')
def simple_label(c):
    return 1

In [ ]:
LFs = []

In [ ]:
from snorkel.lf_helpers import test_LF
from snorkel.viewer import SentenceNgramViewer

labeled = []
for c in session.query(Indicator).filter(Indicator.split == 1).all():
#     print(c)
    if LF_up_down(c) != 0:
        labeled.append(c)
        print(c)
print("Number labeled:", len(labeled))

tp, fp, tn, fn = test_LF(session,  LF_up_down, split=1, annotator_name='gold')
sv = SentenceNgramViewer(fp, session)
sv

In [ ]:
from snorkel.annotations import LabelAnnotator
labeler = LabelAnnotator(lfs=LFs)
np.random.seed(1701)
%time L_train = labeler.apply(split=0)
L_train

In [ ]:
%time L_train = labeler.load_matrix(session, split=0)
L_train

In [ ]:
L_train.get_candidate(session, 0)

In [ ]:
L_train.get_key(session, 0)

In [ ]:
L_train.lf_stats(session)

In [ ]:
from snorkel.learning import GenerativeModel

gen_model = GenerativeModel()
gen_model.train(L_train, epochs=100, decay=0.95, step_size=0.1 / L_train.shape[0], reg_param=1e-6)

gen_model.weights.lf_accuracy

train_marginals = gen_model.marginals(L_train)

In [ ]:
import matplotlib.pyplot as plt
plt.hist(train_marginals, bins=20)
plt.show()

In [ ]:
gen_model.learned_lf_stats()

In [ ]:
L_dev = labeler.apply_existing(split=1)
tp, fp, tn, fn = gen_model.error_analysis(session, L_dev, L_gold_dev)


In [ ]:
from snorkel.viewer import SentenceNgramViewer

# NOTE: This if-then statement is only to avoid opening the viewer during automated testing of this notebook
# You should ignore this!
import os
if 'CI' not in os.environ:
    sv = SentenceNgramViewer(fp, session)
else:
    sv = None
sv

In [ ]:
c = sv.get_selected() if sv else list(fp.union(fn))[0]
c

In [ ]:
c.labels

In [ ]:
L_dev.lf_stats(session, L_gold_dev, gen_model.learned_lf_stats()['Accuracy'])

In [ ]:
from snorkel.annotations import save_marginals
%time save_marginals(session, L_train, train_marginals)